<a href="https://colab.research.google.com/github/mridul-eecs/signal-processing-apachesparkml-apachesystemml/blob/master/GBDT_sparkML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Preprocessing to work with spark

In [1]:

# spark dependencies:
# citation: http://medium.com/@rmache/big-data-with-spark-in-google-colab-7c046e24b3
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apachemirror.wuchna.com/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz

!pip install -q findspark
!pip install pyspark

     |████████████████████████████████| 215.7MB 58kB/s 
     |████████████████████████████████| 204kB 46.5MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.4-py2.py3-none-any.whl size=216130387 sha256=d415f830a6e87ac0700a16d808512662faa1f4b7c037842c995e8de9952f35a5
  Stored in directory: /root/.cache/pip/wheels/ab/09/4d/0d184230058e654eb1b04467dbc1292f00eaa186544604b471
Successfully built pyspark


In [2]:
!git clone https://github.com/mridul-eecs/signal-processing-apachesparkml-apachesystemml.git

Cloning into 'signal-processing-apachesparkml-apachesystemml'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 34 (delta 7), reused 16 (delta 2), pack-reused 0
Unpacking objects: 100% (34/34), done.


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, IntegerType
from pyspark.sql.functions import lit
import os
from tqdm import tqdm_notebook as tqdm

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"


import findspark
findspark.init()

APP_NAME= "Human Motion Premitives"
SPARK_URL= "local[*]"
RANDOM_SEED = 141109
TRAINING_DATA_RATIO = 0.7
RF_NUM_TREES = 8
RF_MAX_DEPTH = 4
RF_NUM_BINS = 32

In [0]:
spark= SparkSession.builder.appName(APP_NAME).master(SPARK_URL).getOrCreate()

### Main program

In [28]:
datapath= '/content/signal-processing-apachesparkml-apachesystemml/df.parquet'
df= spark.read.parquet(datapath)
df= df.createOrReplaceTempView('df')
spark.sql("""select df.class from df""").distinct().collect()

[Row(class='Use_telephone'),
 Row(class='Standup_chair'),
 Row(class='Eat_meat'),
 Row(class='Getup_bed'),
 Row(class='Drink_glass'),
 Row(class='Pour_water'),
 Row(class='Comb_hair'),
 Row(class='Walk'),
 Row(class='Climb_stairs'),
 Row(class='Sitdown_chair'),
 Row(class='Liedown_bed'),
 Row(class='Descend_stairs'),
 Row(class='Brush_teeth'),
 Row(class='Eat_soup')]

In [31]:
binarydf= spark.sql("""select * from df where class in ('Standup_chair', 'Use_telephone')""")
binarydf.show()

+---+---+---+-------------+
|  x|  y|  z|        class|
+---+---+---+-------------+
| 14| 46| 31|Standup_chair|
| 49| 24| 40|Use_telephone|
|  7| 30| 17|Standup_chair|
| 16| 41| 44|Standup_chair|
| 34| 43| 44|Use_telephone|
| 14| 40| 33|Standup_chair|
| 14| 40| 33|Standup_chair|
| 14| 40| 33|Standup_chair|
| 14| 40| 33|Standup_chair|
| 14| 40| 33|Standup_chair|
| 14| 40| 33|Standup_chair|
| 14| 40| 33|Standup_chair|
| 44| 31| 50|Use_telephone|
| 44| 31| 50|Use_telephone|
| 12| 30| 33|Standup_chair|
| 29| 41| 51|Standup_chair|
| 29| 41| 51|Standup_chair|
| 29| 41| 51|Standup_chair|
| 29| 41| 51|Standup_chair|
| 25| 36| 44|Use_telephone|
+---+---+---+-------------+
only showing top 20 rows



In [0]:
train_df, test_df= binarydf.randomSplit([0.8, 0.2])

In [0]:
from pyspark.ml.feature import StringIndexer, VectorAssembler, Normalizer
from pyspark.ml import Pipeline
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

indexer= StringIndexer(inputCol='class', outputCol= 'label')
vectorizer= VectorAssembler(inputCols= ['x', 'y', 'z'], outputCol= 'feats')
normalizer= Normalizer(inputCol= 'feats', outputCol= 'features')
gbdt= GBTClassifier(labelCol= 'label', featuresCol= 'features', maxIter= 10)

eval= MulticlassClassificationEvaluator().setMetricName('accuracy').setPredictionCol('prediction').setLabelCol('label')

pipe= Pipeline(stages= [indexer, vectorizer, normalizer, gbdt])
model= pipe.fit(train_df)
pred_train= model.transform(train_df)

In [48]:
eval.evaluate(pred_train)

0.9113277765816751

In [49]:
pred_test= model.transform(test_df)
eval.evaluate(pred_test)

0.90564645097798